## Module 2 Homework

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---

---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame. 
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**. 
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Holdings” → `Holdings`
   - “Ltd” or “Limited” → `Ltd`
   - Others → `Other`

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`
4. Convert `Shares Offered` to numeric, clean missing or invalid values.
5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)
6. Group by `Company Class` and calculate total withdrawn value.
7. **Answer**: Which class had the highest **total** value of withdrawals?
---

In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt
     

In [2]:

import pandas as pd
import requests
from io import StringIO

def get_ipos(url):
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [3]:
ipo_wdr = get_ipos("https://stockanalysis.com/ipos/withdrawn")

print(ipo_wdr.info())
print(ipo_wdr.isnull().sum())

ipo_wdr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
Symbol            0
Company Name      0
Price Range       0
Shares Offered    0
dtype: int64


,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [4]:
def classify_company(name):
    name = name.lower()
    if "acquisition corp" in name or "acquisition corporation" in name:
        return "Acq.Corp"
    elif "inc" in name or "incorporated" in name:
        return "Inc"
    elif "group" in name:
        return "Group"
    elif "holdings" in name:
        return "Holdings"
    elif "ltd" in name or "limited" in name:
        return "Ltd"
    else:
        return "Other"



import numpy as np
import re

def extract_price(price_range):
    # Handle empty or '--' cases
    if pd.isna(price_range) or price_range == '--':
        return np.nan
    
    # Remove $ and M characters
    cleaned_price = price_range.replace('$', '').replace('M', '')
    
    # Check if it's a range (contains a hyphen)
    if '-' in cleaned_price:
        # Extract the two values from the range
        values = re.findall(r'\d+\.?\d*', cleaned_price)
        if len(values) >= 2:
            # Calculate midpoint
            return (float(values[0]) + float(values[1])) / 2
    else:
        # For absolute values
        values = re.findall(r'\d+\.?\d*', cleaned_price)
        if values:
            return float(values[0])
    
    # Fallback if no numbers found
    return np.nan






In [5]:
ipo_wdr["Company Class"] = ipo_wdr["Company Name"].apply(classify_company)

ipo_wdr["Amount"] = ipo_wdr["Price Range"].apply(extract_price)

ipo_wdr["Shares Offered"] = pd.to_numeric(ipo_wdr["Shares Offered"], errors='coerce')

# Amount is already the average price from your previous step
ipo_wdr["Withdrawn Value"] = ipo_wdr["Shares Offered"] * ipo_wdr["Amount"]

# 3. Group by 'Company Class' and calculate total withdrawn value
class_totals = ipo_wdr.groupby("Company Class")["Withdrawn Value"].sum().sort_values(ascending=False)

# 4. Find the class with the highest total withdrawn value
highest_class = class_totals.index[0]
highest_value = class_totals.iloc[0]

print(f"Class totals (in descending order):\n{class_totals}")
print(f"\nThe class with the highest total value of withdrawals is: {highest_class}")
print(f"Total withdrawn value: {highest_value:,.2f}")

Class totals (in descending order):
Company Class
Acq.Corp    4.021000e+09
Inc         2.257164e+09
Other       7.679200e+08
Ltd         3.217346e+08
Holdings    3.030000e+08
Group       3.378750e+07
Name: Withdrawn Value, dtype: float64

The class with the highest total value of withdrawals is: Acq.Corp
Total withdrawn value: 4,021,000,000.00


### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.


3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?

---

In [7]:
import pandas as pd
import requests
from io import StringIO

def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [20]:
ipos_2024 = get_ipos_by_year(2024)

ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], errors='coerce')

ipos_2024['IPO Price'] = pd.to_numeric(ipos_2024['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
ipos_2024['IPO Price'] = pd.to_numeric(ipos_2024['IPO Price'])

# Convert "Current" column
ipos_2024['Current'] = pd.to_numeric(ipos_2024['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
ipos_2024['Return'] = pd.to_numeric(ipos_2024['Return'].str.replace('%', ''), errors='coerce') / 100

ipos_2024.info()

print(ipos_2024.isnull().sum())

ipos_2024



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      225 non-null    datetime64[ns]
 1   Symbol        225 non-null    object        
 2   Company Name  225 non-null    object        
 3   IPO Price     221 non-null    float64       
 4   Current       225 non-null    float64       
 5   Return        221 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.7+ KB
IPO Date        0
Symbol          0
Company Name    0
IPO Price       4
Current         0
Return          4
dtype: int64


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-12-31,ONEG,OneConstruction Group Limited,4.00,3.45,-0.1375
1,2024-12-27,PHH,"Park Ha Biological Technology Co., Ltd.",4.00,18.70,3.6750
2,2024-12-23,HIT,"Health In Tech, Inc.",4.00,0.58,-0.8544
3,2024-12-23,TDAC,Translational Development Acquisition Corp.,10.00,10.25,0.0248
4,2024-12-20,RANG,Range Capital Acquisition Corp.,10.00,10.45,0.0450
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,6.00,1.10,-0.8167
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,16.45,13.93,-0.1532
222,2024-01-12,SYNX,Silynxcom Ltd.,4.00,1.72,-0.5700
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,21.00,20.19,-0.0386


In [25]:
ipo_tickers = ipos_2024['Symbol'].dropna().unique()
ipo_tickers

array(['ONEG', 'PHH', 'HIT', 'TDAC', 'RANG', 'YAAS', 'LSE', 'NCEW',
       'YSXT', 'AVR', 'DRDB', 'MLAC', 'NTCL', 'TTAN', 'JACS', 'NAMI',
       'LNKS', 'SPHA', 'ZSPC', 'TAVI', 'JUNS', 'BRIA', 'GRO', 'PONY',
       'VENU', 'CGTL', 'FACT', 'WYHG', 'CASK', 'MSW', 'DYNX', 'NOEM',
       'HSPT', 'IZTC', 'ASPC', 'WLAC', 'GSRT', 'BACQ', 'NTWO', 'GELS',
       'PLRZ', 'SEPN', 'WRD', 'BEAG', 'CHAR', 'INGM', 'OACC', 'FGL',
       'LBGJ', 'ROLR', 'SAG', 'ALDF', 'HUHU', 'ORIS', 'SPHL', 'PTLE',
       'SFHG', 'CAMP', 'CBLL', 'CCIR', 'RDAC', 'STFS', 'UPB', 'CLIK',
       'KLC', 'LPBB', 'SKK', 'CBNA', 'FLX', 'FVR', 'SARO', 'WCT', 'HTLM',
       'ZJK', 'BIOA', 'BKV', 'GRDN', 'IBG', 'LGCY', 'PC', 'CAPN', 'GLE',
       'YHNA', 'IBO', 'KAPA', 'BCAX', 'MBX', 'ZBIO', 'FVN', 'GLXG',
       'TDTH', 'XCH', 'POLE', 'VCIC', 'PMAX', 'PTHL', 'GIG', 'AZI',
       'BSII', 'SPAI', 'JBDI', 'RITR', 'WOK', 'HOND', 'SBXD', 'YXT',
       'ACTU', 'CEP', 'VACH', 'EQV', 'AAM', 'MBAV', 'OSTX', 'NIPG', 'CON',
       'DTSQ', 

In [26]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ipo_tickers):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-5) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 ONEG
1 PHH
2 HIT
3 TDAC
4 RANG
5 YAAS
6 LSE
7 NCEW
8 YSXT
9 AVR
10 DRDB
11 MLAC
12 NTCL
13 TTAN
14 JACS
15 NAMI
16 LNKS
17 SPHA


HTTPError: HTTP Error 404: 

In [30]:
for i, ticker in enumerate(ipo_tickers):
    print(i, ticker)

    try:
        ticker_obj = yf.Ticker(ticker)
        historyPrices = ticker_obj.history(period="max", interval="1d")

        if historyPrices.empty:
            print(f"Warning: No data found for {ticker}")
            continue

        # Continue processing as before...

        # Add your columns/features
        historyPrices['Ticker'] = ticker
        historyPrices['Year'] = historyPrices.index.year
        historyPrices['Month'] = historyPrices.index.month
        historyPrices['Weekday'] = historyPrices.index.weekday
        historyPrices['Date'] = historyPrices.index.date

        for j in [1, 3, 7, 30, 90, 252, 365]:
            historyPrices[f'growth_{j}d'] = historyPrices['Close'] / historyPrices['Close'].shift(j)

        historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']
        historyPrices['SMA10'] = historyPrices['Close'].rolling(10).mean()
        historyPrices['SMA20'] = historyPrices['Close'].rolling(20).mean()
        historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
        historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']
        historyPrices['volatility'] = historyPrices['Close'].rolling(30).std() * np.sqrt(252)
        historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

        if stocks_df.empty:
            stocks_df = historyPrices
        else:
            stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

    except Exception as e:
        print(f"Error for ticker {ticker}: {e}")
    
    time.sleep(1)


0 ONEG


1 PHH
2 HIT
3 TDAC
4 RANG
5 YAAS
6 LSE
7 NCEW
8 YSXT
9 AVR
10 DRDB
11 MLAC
12 NTCL
13 TTAN
14 JACS
15 NAMI
16 LNKS


$SPHA: possibly delisted; no timezone found


17 SPHA
18 ZSPC
19 TAVI
20 JUNS
21 BRIA
22 GRO
23 PONY
24 VENU
25 CGTL
26 FACT
27 WYHG
28 CASK
29 MSW
30 DYNX
31 NOEM
32 HSPT


$IZTC: possibly delisted; no timezone found


33 IZTC
34 ASPC
35 WLAC
36 GSRT
37 BACQ
38 NTWO
39 GELS
40 PLRZ
41 SEPN
42 WRD
43 BEAG
44 CHAR
45 INGM
46 OACC
47 FGL
48 LBGJ
49 ROLR
50 SAG
51 ALDF
52 HUHU
53 ORIS
54 SPHL
55 PTLE
56 SFHG
57 CAMP
58 CBLL
59 CCIR
60 RDAC
61 STFS
62 UPB
63 CLIK
64 KLC
65 LPBB
66 SKK
67 CBNA
68 FLX
69 FVR
70 SARO
71 WCT
72 HTLM
73 ZJK
74 BIOA
75 BKV
76 GRDN
77 IBG
78 LGCY
79 PC
80 CAPN
81 GLE
82 YHNA
83 IBO
84 KAPA
85 BCAX
86 MBX
87 ZBIO
88 FVN
89 GLXG
90 TDTH
91 XCH
92 POLE
93 VCIC
94 PMAX
95 PTHL
96 GIG
97 AZI
98 BSII
99 SPAI
100 JBDI
101 RITR
102 WOK
103 HOND
104 SBXD
105 YXT
106 ACTU
107 CEP
108 VACH
109 EQV
110 AAM
111 MBAV
112 OSTX
113 NIPG
114 CON
115 DTSQ
116 LINE
117 ORKT
118 BLMZ
119 OS
120 ZDAI
121 ARTV
122 QMMM
123 ARDT
124 TWFG
125 ICON
126 LPAA
127 SIMA
128 EHGO
129 EURK
130 ALMS
131 LB
132 LSH
133 TBN
134 WBTN
135 GRAF
136 RECT
137 CUB
138 FSHP
139 MACI
140 TEM
141 PCSC
142 ALF
143 CHEB
144 RAPP
145 WAY
146 FLYE
147 GAUZ
148 NAKA
149 BOW
150 HDL
151 RFAI
152 JDZG
153 RAY
154 BTOC
155 ZK
15

In [43]:

stocks_df.groupby(['Ticker']).Date.agg(['min','max'])

,min,max
Ticker,,
AAM,2024-09-16,2025-06-10
ACTU,2024-08-13,2025-06-10
AHR,2024-02-07,2025-06-10
ALAB,2024-03-20,2025-06-10
ALDF,2024-12-16,2025-06-10
...,...,...
ZDAI,2024-07-23,2025-06-10
ZJK,2024-09-30,2025-06-10
ZK,2024-05-10,2025-06-10


In [ ]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']




,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe


In [57]:
from datetime import date

# Define the date you're interested in
specific_date = date(2025, 6, 6)

# Filter the DataFrame
filtered_df = stocks_df[stocks_df['Date'] == specific_date]
filtered_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe
107,3.540,3.59,3.320,3.510,8400,0.0,0.0,ONEG,2025,6,...,NaN,NaN,NaN,3.3600,3.30790,1,0.076923,10.776806,0,NaN
219,16.760,18.35,16.000,17.840,337300,0.0,0.0,PHH,2025,6,...,NaN,NaN,NaN,14.8775,14.51325,1,0.131726,15.241719,0,NaN
334,0.635,0.65,0.614,0.630,59300,0.0,0.0,HIT,2025,6,...,NaN,NaN,NaN,0.6401,0.67890,0,0.057143,1.151785,0,NaN
415,10.210,10.21,10.210,10.210,0,0.0,0.0,TDAC,2025,6,...,NaN,NaN,NaN,10.2010,10.19400,1,0.000000,0.416224,0,NaN
517,10.250,10.25,10.250,10.250,0,0.0,0.0,RANG,2025,6,...,NaN,NaN,NaN,10.1760,10.15400,1,0.000000,0.948405,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103606,1.075,1.10,1.075,1.095,1600,0.0,0.0,CCTG,2025,6,...,0.500000,NaN,NaN,1.1434,1.18080,0,0.022831,1.297728,0,0.350613
103956,13.900,14.00,13.750,13.880,56200,0.0,0.0,PSBD,2025,6,...,0.947565,NaN,NaN,13.9030,13.74250,1,0.018012,9.712591,0,0.092927
104309,1.750,1.75,1.680,1.680,10000,0.0,0.0,SYNX,2025,6,...,0.626632,NaN,NaN,1.7435,1.82090,0,0.041667,1.368371,0,0.425054
104661,19.900,20.15,19.270,19.270,38100,0.0,0.0,SDHC,2025,6,...,0.758065,NaN,NaN,18.6850,18.66500,1,0.045667,16.442821,0,0.043366


In [58]:
filtered_df[['Sharpe','growth_252d']].describe()

,Sharpe,growth_252d
count,146.000000,146.000000
mean,0.297523,1.227946
std,0.521383,1.475125
min,-0.079677,0.024970
25%,0.040265,0.293510
50%,0.083768,0.763188
75%,0.331967,1.446667
max,2.835668,8.097413


In [73]:
f"Median of sharpe is {float(filtered_df['Sharpe'].median())*100:.1f}%"

'Median of sharpe is 8.4%'

### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).

